# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys

#reference folder key file is in
sys.path.insert(1, os.path.abspath('../WeatherPy/'))

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#upload weather data and load into dataframe
file = "../WeatherPy/cityweatherdata.csv"
weather_df = pd.read_csv(file)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,magadan,59.5667,150.8000,33.62,88,99,28.34,RU,1618911179
1,talaja,21.3500,72.0500,93.20,40,75,14.97,IN,1618911390
2,nikolskoye,59.7035,30.7861,52.00,25,0,13.42,RU,1618911390
3,troitskoye,52.9821,84.6757,26.47,55,100,7.23,RU,1618911391
4,panacan,9.2463,118.4082,87.80,62,20,6.91,PH,1618911391


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
#configure gmaps key
gmaps.configure(api_key=g_key)

#set and display heatmap of humidity
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    weather_df[['Lat','Lng']], weights = weather_df['Humidity'],
    max_intensity=weather_df['Humidity'].max()
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
#sorting ideal weather dataframe

ideal_weather = weather_df.query('`Max Temp`<80 & `Max Temp`>70 & `Wind Speed`<10 &Cloudiness==0')
ideal_weather


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
229,saint-francois,16.2526,-61.2741,75.20,83,0,4.61,GP,1618911439
235,boa vista,2.8197,-60.6733,75.20,83,0,5.75,BR,1618911435
274,alice springs,-23.7000,133.8833,73.99,17,0,8.05,AU,1618911213
323,menongue,-14.6585,17.6910,78.06,38,0,9.35,AO,1618911459
345,cape town,-33.9258,18.4232,72.00,53,0,8.05,ZA,1618911286
360,yulara,-25.2406,130.9889,73.40,21,0,8.05,AU,1618911213
375,hervey bay,-25.2986,152.8535,73.99,65,0,3.00,AU,1618911469
440,maceio,-9.6658,-35.7353,73.40,94,0,4.61,BR,1618911461
442,tiarei,-17.5333,-149.3333,78.80,78,0,3.44,PF,1618911482
446,mahina,-17.5065,-149.4890,78.80,78,0,3.44,PF,1618911483


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [53]:
#empty hotel_df
hotel_df = pd.DataFrame(columns = ["Hotel Name"])

In [54]:
#set base url plus params dictionary
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "radius":5000,
    "keyword": "hotel",
    "type":"lodging",
    "key": g_key,
}

In [55]:
#loop through ideal weather cities
for index, row in ideal_weather.iterrows():


# add location to params dict
    params['location'] = f"{str(row['Lat'])},{str(row['Lng'])}"


# make url and make API request
    response = requests.get(base_url, params=params).json()
    
# extract results
    results = response['results']

#fill in hotel_df with hotel information
    try:
       
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']
        hotel_df.loc[index, 'City'] = row['City']
        hotel_df.loc[index, 'Country'] = row['Country']       
        
    except:
        print(f"No Hotels near {row['City']}")


No Hotels near tautira


In [57]:
#display hotels
hotel_df

,Hotel Name,Lat,Lng,City,Country
229,Villas Boubou location villas luxe Guadeloupe,16.254803,-61.259798,saint-francois,GP
235,Hotel Orla do Rio Branco,2.808403,-60.670938,boa vista,BR
274,DoubleTree by Hilton Hotel Alice Springs,-23.715016,133.879153,alice springs,AU
323,Hotel Ritz Lauca,-14.659533,17.673109,menongue,AO
345,Atlanticview Cape Town Boutique Hotel,-33.959609,18.385298,cape town,ZA
360,Sails in the Desert,-25.239022,130.983405,yulara,AU
375,Oaks Hervey Bay Resort and Spa,-25.282624,152.901037,hervey bay,AU
440,Jatiúca Hotel & Resort,-9.641322,-35.699256,maceio,BR
442,Le Rocher de Tahiti,-17.539562,-149.366494,tiarei,PF
446,Le Tahiti by Pearl Resorts,-17.518109,-149.506609,mahina,PF


In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [59]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))